In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import random
import numpy as np
import seaborn as sns
import math
import os
import tempfile
import matplotlib as mpl
from sklearn.metrics import confusion_matrix



In [2]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
#Karpathy 1. fix random seed
tf.random.set_seed(42)
random.seed(42)
np.random.seed(42)

In [4]:
ATT_FILE_NAME = "FootballPlayerPreparedCleanAttributes.csv"
ONE_HOT_ENCODED_CLASSES_FILE_NAME = "FootballPlayerOneHotEncodedClasses.csv"

In [5]:
att = pd.read_csv(ATT_FILE_NAME)
target = pd.read_csv(ONE_HOT_ENCODED_CLASSES_FILE_NAME)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
att.describe()

,Crossing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Reactions,ShotPower,Stamina,LongShots,Aggression,Positioning,Vision,Composure
count,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000
mean,0.053537,0.064942,0.168294,-0.081373,0.122654,-0.030012,-0.132537,0.002751,0.076091,0.098800,0.125965,0.169170,-0.025506,0.136145,0.043417,0.060219,0.036497
std,0.344606,0.293820,0.268085,0.369549,0.301492,0.364944,0.358826,0.334822,0.282066,0.233806,0.326638,0.324097,0.379027,0.352347,0.349409,0.314939,0.274291
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-0.195122,-0.139241,0.013699,-0.375000,-0.012048,-0.325301,-0.428571,-0.189189,-0.070423,-0.066667,-0.086420,-0.014493,-0.301205,-0.097561,-0.166667,-0.146341,-0.162162
50%,0.097561,0.088608,0.205479,-0.075000,0.180723,-0.012048,-0.190476,0.054054,0.098592,0.093333,0.160494,0.188406,0.036145,0.170732,0.095238,0.097561,0.054054
75%,0.317073,0.265823,0.342466,0.200000,0.325301,0.253012,0.142857,0.243243,0.267606,0.253333,0.358025,0.391304,0.277108,0.390244,0.285714,0.292683,0.243243
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
target.head()

,"Poor:[46.0, 62.0]","Interm:[63.0, 66.0]","Good:[67.0, 71.0]","Excel:[72.0, 94.0]"
0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0


In [9]:
clases = np.argmax(target.to_numpy(),axis=1)

In [10]:
np.bincount(clases) # contar cuantos elemntos tengo de cada clase en el conjunto de datos

array([4296, 3868, 4353, 3605], dtype=int64)

In [11]:
c1, c2,c3,c4= np.bincount(clases)
total = c1+c2+c3+c4
print(f"Total: {total}. C1 {c1}, c2 {c2}, c3 {c3}, c4 {c4}") #lo mismo de arriba

Total: 16122. C1 4296, c2 3868, c3 4353, c4 3605


In [12]:
X_train,X_rem,y_train,y_rem = train_test_split(att,target,train_size=0.8)
X_valid,X_test,y_valid,y_test = train_test_split(X_rem,y_rem,train_size=0.5)
#80 % train   --- 10% val 10% test

In [13]:
c1, c2,c3,c4= np.bincount(np.argmax(y_train.to_numpy(),axis=1))
total = c1+c2+c3+c4
print(f"Total: {total}. C1 {c1}, c2 {c2}, c3 {c3}, c4 {c4}")

Total: 12897. C1 3455, c2 3086, c3 3479, c4 2877


In [14]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias) # inicializar el bias de la ultima capa de manera sesgada para que todas las clases sean "casi" iguales
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(17,)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = make_model()
model.summary()

In [ ]:
-math.log(1/4)

In [ ]:
results = model.evaluate(X_train,y_train,batch_size=BATCH_SIZE)
print(f"Loss: {results[0]}")

In [ ]:
frequency = [c1/total,c2/total,c3/total,c4/total]
frequency


In [ ]:
bi = np.log(frequency)
bi

In [ ]:
from scipy.optimize import fsolve
from math import exp
def eqn(x, frequency=frequency):
  sum_exp = sum([exp(x_i) for x_i in x])
  return [exp(x[i])/sum_exp - frequency[i] for i in range(len(frequency))]

# calculate bias init
bias_init = fsolve(func=eqn,
                  x0=[0]*len(frequency),
                  ).tolist()

bias_init 

In [ ]:
model = make_model(output_bias=bias_init)
model.evaluate(X_train,y_train,batch_size=BATCH_SIZE)

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [ ]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0,0.0,0.0,0.0])
zero_bias_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(X_test, y_test), 
    verbose=1)

In [ ]:
model = make_model(output_bias=bias_init)
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(X_test, y_test), 
    verbose=1)

In [ ]:
def plot_loss(history, label, n):
  # Use a log scale on y-axis to show the wide range of values.
  plt.semilogy(history.epoch, history.history['loss'],
               color=colors[n], label='Train ' + label)
  plt.semilogy(history.epoch, history.history['val_loss'],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')

In [ ]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_metrics(careful_bias_history)


In [ ]:
train_predictions_baseline = model.predict(X_train, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(X_test, batch_size=BATCH_SIZE)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels,predictions)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix')
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')


In [ ]:
baseline_results = model.evaluate(X_test, y_test,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(np.argmax(y_test.to_numpy(),axis=1), test_predictions_baseline.argmax(axis=1))

Generación de un modelo simple con un perceptron de una capa y pocas neuronas para conocer un poco donde estamos.

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
   
    #keras.layers.Dense(10, activation='relu'),

    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)
model.summary()
h_nocapas = model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Utilizando 1 capa y pocas neuronas

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(10, activation='elu'),

    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_unacapa =model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Utilizando dos capas

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(10, activation='elu'),
    keras.layers.Dense(10, activation='elu'),


    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_doscapas =model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Observamos que a mas capas, mejores resultados. Verificación utilizando muchas neuronas por capa

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(128, activation='elu'),
    keras.layers.Dense(128, activation='elu'),


    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_128128= model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Resultados mejores. Probamos con aún más neuronas.

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(512, activation='elu'),
    keras.layers.Dense(512, activation='elu'),


    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_512512= model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Añadir más neuronas auemnta el tiempo de entrenamiento ligeramente (CPU only) sin dar mejores resultados

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(512, activation='elu'),
    keras.layers.Dense(512, activation='elu'),
    keras.layers.Dense(512, activation='elu'),
    keras.layers.Dense(512, activation='elu'),




    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_512x4= model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

El objetivo ahora es conseguir un modelo lo bastante grande que haga overfit a los datos y reducirlo

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),




    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_100x4x20= model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test)) #menos epoch

Como la precisión no parece estancarse lo dejamos durante muchas mas epocas

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),




    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_100x4x100= model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Observamos el overfit del modelo

In [ ]:
plt.plot(h_100x4x100.history['val_categorical_accuracy'])
plt.plot(h_100x4x100.history['categorical_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['val_accuracy','train_accuracy'], loc='upper left')
plt.show()
# "Loss"

Aplicamos estrategias de regularización.

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),





    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_100x4x100= model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Tiene buana pinta

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),





    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_100x4x100= model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Añadir una capa mas no aporta nada

In [ ]:
model = model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,)),
    
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),

    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),
    





    keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()
h_100x4x100= model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

In [ ]:
plt.plot(h_128128.history['val_categorical_accuracy'])
plt.plot(h_nocapas.history['val_categorical_accuracy'])
plt.plot(h_512512.history['val_categorical_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['h28128', 'nocapas','512512'], loc='upper left')
plt.show()
# "Loss"


In [18]:
def build_model(n_hidden=1, n_neurons_ocultas=[30], learning_rate=3e-3, input_shape=(8,), 
                activation_o="relu", optimizer="adam", loss="categorical_crossentropy", metrics=METRICS, 
                activation_s="soft_max", n_neurons_salida=4):     
    model = keras.models.Sequential()     
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    if n_hidden != len(n_neurons_ocultas):
        print("error, tamaño de la lista de n_neuronas no corresponde al umero de capas")
        return -1
    for layer in range(n_hidden):         
        model.add(keras.layers.Dense(n_neurons_ocultas[layer], activation="relu"))     
  
    model.add(keras.layers.Dense(n_neurons_salida, activation=activation_s))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)     
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)    
    return model

In [19]:
model = build_model(2, [30,20], 0.5, (17,), "relu", "adam", "categorical_crossentropy", METRICS, "softmax", 4)


model.summary()
h_100x4x100= model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test), batch_size=256)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                540       
                                                                 
 dense_4 (Dense)             (None, 20)                620       
                                                                 
 dense_5 (Dense)             (None, 4)                 84        
                                                                 
Total params: 1,244
Trainable params: 1,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 2s 14ms/step - loss: 0.9303 - tp: 10271.0000 - fp: 2929.0000 - tn: 59954.0000 - fn: 10690.0000 - accuracy: 0.8376 - precision: 0.7781 - recall: 0.4900 - auc: 0.8964 - prc: 0.7570 - val_loss: 0.7535 - val_tp: 922.0000 - val_fp: 418.0000 - val_tn: 4421.0000 - val_f

51/51 [==============================] - 0s 3ms/step - loss: 0.4566 - tp: 10209.0000 - fp: 2545.0000 - tn: 36146.0000 - fn: 2688.0000 - accuracy: 0.8986 - precision: 0.8005 - recall: 0.7916 - auc: 0.9597 - prc: 0.8897 - val_loss: 0.4873 - val_tp: 1260.0000 - val_fp: 343.0000 - val_tn: 4496.0000 - val_fn: 353.0000 - val_accuracy: 0.8921 - val_precision: 0.7860 - val_recall: 0.7812 - val_auc: 0.9541 - val_prc: 0.8745
Epoch 19/100
51/51 [==============================] - 0s 3ms/step - loss: 0.4574 - tp: 10238.0000 - fp: 2545.0000 - tn: 36146.0000 - fn: 2659.0000 - accuracy: 0.8991 - precision: 0.8009 - recall: 0.7938 - auc: 0.9596 - prc: 0.8892 - val_loss: 0.6249 - val_tp: 1136.0000 - val_fp: 442.0000 - val_tn: 4397.0000 - val_fn: 477.0000 - val_accuracy: 0.8576 - val_precision: 0.7199 - val_recall: 0.7043 - val_auc: 0.9302 - val_prc: 0.8174
Epoch 20/100
51/51 [==============================] - 0s 3ms/step - loss: 0.4606 - tp: 10171.0000 - fp: 2597.0000 - tn: 36094.0000 - fn: 2726.0000 - 

Epoch 37/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4300 - tp: 10405.0000 - fp: 2383.0000 - tn: 36308.0000 - fn: 2492.0000 - accuracy: 0.9055 - precision: 0.8137 - recall: 0.8068 - auc: 0.9639 - prc: 0.9004 - val_loss: 0.4378 - val_tp: 1290.0000 - val_fp: 302.0000 - val_tn: 4537.0000 - val_fn: 323.0000 - val_accuracy: 0.9031 - val_precision: 0.8103 - val_recall: 0.7998 - val_auc: 0.9626 - val_prc: 0.8969
Epoch 38/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4368 - tp: 10362.0000 - fp: 2431.0000 - tn: 36260.0000 - fn: 2535.0000 - accuracy: 0.9037 - precision: 0.8100 - recall: 0.8034 - auc: 0.9630 - prc: 0.8978 - val_loss: 0.4641 - val_tp: 1267.0000 - val_fp: 341.0000 - val_tn: 4498.0000 - val_fn: 346.0000 - val_accuracy: 0.8935 - val_precision: 0.7879 - val_recall: 0.7855 - val_auc: 0.9581 - val_prc: 0.8839
Epoch 39/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4214 - tp: 10465.0000 - fp: 2324.0000 - tn: 36367.0000 - fn:

Epoch 56/100
51/51 [==============================] - 0s 3ms/step - loss: 0.4217 - tp: 10472.0000 - fp: 2335.0000 - tn: 36356.0000 - fn: 2425.0000 - accuracy: 0.9077 - precision: 0.8177 - recall: 0.8120 - auc: 0.9654 - prc: 0.9046 - val_loss: 0.4470 - val_tp: 1291.0000 - val_fp: 303.0000 - val_tn: 4536.0000 - val_fn: 322.0000 - val_accuracy: 0.9031 - val_precision: 0.8099 - val_recall: 0.8004 - val_auc: 0.9613 - val_prc: 0.8951
Epoch 57/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4102 - tp: 10514.0000 - fp: 2283.0000 - tn: 36408.0000 - fn: 2383.0000 - accuracy: 0.9096 - precision: 0.8216 - recall: 0.8152 - auc: 0.9673 - prc: 0.9098 - val_loss: 0.4259 - val_tp: 1315.0000 - val_fp: 292.0000 - val_tn: 4547.0000 - val_fn: 298.0000 - val_accuracy: 0.9086 - val_precision: 0.8183 - val_recall: 0.8153 - val_auc: 0.9648 - val_prc: 0.9032
Epoch 58/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4098 - tp: 10526.0000 - fp: 2290.0000 - tn: 36401.0000 - fn:

Epoch 75/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4017 - tp: 10624.0000 - fp: 2189.0000 - tn: 36502.0000 - fn: 2273.0000 - accuracy: 0.9135 - precision: 0.8292 - recall: 0.8238 - auc: 0.9687 - prc: 0.9134 - val_loss: 0.4418 - val_tp: 1300.0000 - val_fp: 301.0000 - val_tn: 4538.0000 - val_fn: 313.0000 - val_accuracy: 0.9048 - val_precision: 0.8120 - val_recall: 0.8060 - val_auc: 0.9627 - val_prc: 0.8979
Epoch 76/100
51/51 [==============================] - 0s 3ms/step - loss: 0.3988 - tp: 10613.0000 - fp: 2206.0000 - tn: 36485.0000 - fn: 2284.0000 - accuracy: 0.9130 - precision: 0.8279 - recall: 0.8229 - auc: 0.9691 - prc: 0.9145 - val_loss: 0.4186 - val_tp: 1321.0000 - val_fp: 285.0000 - val_tn: 4554.0000 - val_fn: 292.0000 - val_accuracy: 0.9106 - val_precision: 0.8225 - val_recall: 0.8190 - val_auc: 0.9664 - val_prc: 0.9087
Epoch 77/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4033 - tp: 10606.0000 - fp: 2202.0000 - tn: 36489.0000 - fn:

Epoch 94/100
51/51 [==============================] - 0s 2ms/step - loss: 0.3966 - tp: 10646.0000 - fp: 2181.0000 - tn: 36510.0000 - fn: 2251.0000 - accuracy: 0.9141 - precision: 0.8300 - recall: 0.8255 - auc: 0.9693 - prc: 0.9153 - val_loss: 0.4295 - val_tp: 1306.0000 - val_fp: 295.0000 - val_tn: 4544.0000 - val_fn: 307.0000 - val_accuracy: 0.9067 - val_precision: 0.8157 - val_recall: 0.8097 - val_auc: 0.9647 - val_prc: 0.9048
Epoch 95/100
51/51 [==============================] - 0s 3ms/step - loss: 0.3975 - tp: 10602.0000 - fp: 2210.0000 - tn: 36481.0000 - fn: 2295.0000 - accuracy: 0.9127 - precision: 0.8275 - recall: 0.8221 - auc: 0.9693 - prc: 0.9149 - val_loss: 0.4669 - val_tp: 1277.0000 - val_fp: 316.0000 - val_tn: 4523.0000 - val_fn: 336.0000 - val_accuracy: 0.8989 - val_precision: 0.8016 - val_recall: 0.7917 - val_auc: 0.9595 - val_prc: 0.8913
Epoch 96/100
51/51 [==============================] - 0s 2ms/step - loss: 0.3909 - tp: 10621.0000 - fp: 2186.0000 - tn: 36505.0000 - fn:

IGNORAR NLP CON TORCH

In [ ]:
import numpy as np

In [ ]:
att.columns[2]

In [ ]:
cadenas = []
cadenas_target = []
for x,y in zip(att.to_numpy(),target.to_numpy()):
    temp = ['A football player with']
    for i in range(0,len(x)):
        temp.append(f' {att.columns[i]} of '+str(x[i])+',')
    cadenas.append((''.join(temp)[:-1]))
    cadenas_target.append(np.argmax(y,axis=0))

In [ ]:
cadenas

In [ ]:
datos = []
for x,y in zip(cadenas,cadenas_target):


    misdatos  = {}
    misdatos['label'] = y
    misdatos['text'] = x
    datos.append(misdatos)


In [ ]:
df = pd.DataFrame(data=np.reshape(cadenas,(-1,1)),columns=['text'])

In [ ]:
df['label'] = np.reshape(cadenas_target,(-1,1))

In [ ]:
from datasets import Dataset
ds = Dataset.from_pandas(df)
ds = ds.with_format('torch')
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [ ]:
ds = ds.map(preprocess_function, batched=True)


In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
ds = ds.train_test_split(0.3)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()